In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [51]:
"""
Adapted from https://simpy.readthedocs.io/en/latest/examples/bank_renege.html
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""
import random

import simpy
import simpy.rt
    
def start_simulation(RANDOM_SEED = 1098,
                     NEW_JOB = 250,  # Total number of customers
                     INTERVAL_JOB = 1.0,  # Generate new customers roughly every x seconds
                     PATIENCE = 100000000,  # customer patience
                     TIME_IN_JOB = 1.0,
                     RANDOM_TYPE = True,
                     REALTIME = 0.5,
                     VERBOSE = True):
    # Setup and start the simulation
    def source(env, number, interval, counter, verbose):
        """Source generates customers randomly"""
        env.work_time = []
        env.wait_time = []
        for i in range(number):
            c = customer(env, 'Task%02d' % i, counter, time_in_bank=TIME_IN_JOB, verbose=verbose)
            env.process(c)
            if(RANDOM_TYPE):
                t = random.expovariate(1.0 / interval)
            else:
                t = interval

            yield env.timeout(t)
            

    def customer(env, name, counter, time_in_bank, verbose=True):
        """Customer arrives, is served and leaves."""
        arrive = env.now
        if(verbose):

            print('%7.4f %s: New task arrive' % (arrive, name))

        with counter.request() as req:
            patience = PATIENCE
            # Wait for the counter or abort at the end of our tether
            results = yield req | env.timeout(patience)

            wait = env.now - arrive
            env.wait_time.append(wait)
        

            if req in results:
                # We got to the counter
                if(verbose):
                    print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
                if(RANDOM_TYPE):
                    tib = random.expovariate(1.0 / time_in_bank)
                else:
                    tib = time_in_bank
                env.work_time.append(tib)
                yield env.timeout(tib)
                if(verbose):
                    print('%7.4f %s: Finished, task took %7.2f' % (env.now, name, tib))

            else:
                # We reneged
                if(verbose):
                    print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))

    print('Task Simulation')
    random.seed(RANDOM_SEED)

    if(REALTIME is not None):
        env = simpy.rt.RealtimeEnvironment(factor=REALTIME)
    else:
        env = simpy.Environment()

    # Start processes and run
    counter = simpy.Resource(env, capacity=1)
    proc = env.process(source(env, NEW_JOB, INTERVAL_JOB, counter, VERBOSE))
    env.run(until=proc)

    
    print("Simulated number of tasks: %d" %NEW_JOB)
    print("Average task length: %7.2f" % (sum(env.work_time)/NEW_JOB))
    print("Average wait time: %7.2f" % (sum(env.wait_time)/NEW_JOB))
    print("Average total time: %7.2f " % ((sum(env.wait_time)+sum(env.work_time))/NEW_JOB))
    print("Useful Time = Task time / total time: %2.2f %%" % (sum(env.work_time)/(sum(env.wait_time)+sum(env.work_time))*100))
    print("*Total time taken* %7.2f" % float(env.now))
    print("Period Busy %2.2f %%" % (float(sum(env.work_time))/float(env.now)*100))

In [53]:
##
# CASE 1
start_simulation(NEW_JOB = 30,  # Total number of customers
                 INTERVAL_JOB = 1.0,  # Generate new customers roughly every x seconds
                 TIME_IN_JOB = 1.0,
                 RANDOM_TYPE = False,
                 VERBOSE = True
                 )

Task Simulation
 0.0000 Task00: New task arrive
 0.0000 Task00: Waited  0.000
 1.0000 Task01: New task arrive
 1.0000 Task00: Finished, task took    1.00
 1.0000 Task01: Waited  0.000
 2.0000 Task02: New task arrive
 2.0000 Task01: Finished, task took    1.00
 2.0000 Task02: Waited  0.000
 3.0000 Task03: New task arrive
 3.0000 Task02: Finished, task took    1.00
 3.0000 Task03: Waited  0.000
 4.0000 Task04: New task arrive
 4.0000 Task03: Finished, task took    1.00
 4.0000 Task04: Waited  0.000
 5.0000 Task05: New task arrive
 5.0000 Task04: Finished, task took    1.00
 5.0000 Task05: Waited  0.000
 6.0000 Task06: New task arrive
 6.0000 Task05: Finished, task took    1.00
 6.0000 Task06: Waited  0.000
 7.0000 Task07: New task arrive
 7.0000 Task06: Finished, task took    1.00
 7.0000 Task07: Waited  0.000
 8.0000 Task08: New task arrive
 8.0000 Task07: Finished, task took    1.00
 8.0000 Task08: Waited  0.000
 9.0000 Task09: New task arrive
 9.0000 Task08: Finished, task took    1.0

In [54]:
start_simulation(NEW_JOB = 30,  # Total number of customers
                 INTERVAL_JOB = 1.0,  # Generate new customers roughly every x seconds
                 TIME_IN_JOB = 1.0,
                 RANDOM_TYPE = True,
                 VERBOSE = True
                 )

Task Simulation
 0.0000 Task00: New task arrive
 0.0000 Task00: Waited  0.000
 0.1943 Task01: New task arrive
 0.5562 Task02: New task arrive
 0.7561 Task00: Finished, task took    0.76
 0.7561 Task01: Waited  0.562
 3.7696 Task01: Finished, task took    3.01
 3.7696 Task02: Waited  3.213
 4.3280 Task02: Finished, task took    0.56
 4.3699 Task03: New task arrive
 4.3699 Task03: Waited  0.000
 5.7760 Task03: Finished, task took    1.41
 6.0717 Task04: New task arrive
 6.0717 Task04: Waited  0.000
 6.1837 Task04: Finished, task took    0.11
 7.0858 Task05: New task arrive
 7.0858 Task05: Waited  0.000
 7.3468 Task05: Finished, task took    0.26
 8.0326 Task06: New task arrive
 8.0326 Task06: Waited  0.000
 8.1164 Task07: New task arrive
 9.8518 Task08: New task arrive
10.5015 Task09: New task arrive
10.9965 Task10: New task arrive
12.0365 Task11: New task arrive
12.4884 Task06: Finished, task took    4.46
12.4884 Task07: Waited  4.372
12.7676 Task07: Finished, task took    0.28
12.7676 

In [45]:
interact(start_simulation, 
         RANDOM_SEED=fixed(1098),
         INTERVAL_JOB=(0,2.0,0.1), 
         NEW_JOB=10000,
         TIME_IN_JOB=(0,2.0,0.01),
         RANDOM_TYPE=False,
         VERBOSE=False,
         PATIENCE=fixed(100000000),
         REALTIME=fixed(None))

Task Simulation
Simulated number of tasks: 10000
Average task length:    1.00
Average wait time:    0.00
Average total time:    1.00 
Useful Time = Task time / total time:  100.00 %
*Total time taken* 10000.00
Period Busy 100.00 %
